# Pipeline Quantitativo: Puxar, Processar, Enviar

In [1]:
import pandas as pd
import os
from pathlib import Path
import shutil

## Puxar Dados

In [2]:
import yaml

def load_config(config_path):
    with open(config_path, 'r') as f:
        return yaml.safe_load(f)

config = load_config('../config/config.yml')
assets = config['market']['assets']
data_dir = Path('..') / config['data']['path_raw'] / 'eod'

# Carregar todos os parquets em um dicionário de DataFrames
parquets = {}
loaded_count = 0
for asset in assets:
    parquet_path = data_dir / f"{asset}.parquet"
    if parquet_path.exists():
        parquets[asset] = pd.read_parquet(parquet_path)

## Padronizar Dados

In [3]:
# Consolidar todos os dados em um único DataFrame
df_pre_processado = []
for ticker, df in parquets.items():
    df = df.copy()
    df = df.reset_index()
    df['ticker'] = ticker
    df_pre_processado.append(df)

if df_pre_processado:
    df_consolidado = pd.concat(df_pre_processado, ignore_index=True)
    print(f"Total de registros: {len(df_consolidado)}")
    print(f"Período: {df_consolidado['date'].min().date()} até {df_consolidado['date'].max().date()}")
    print(f"Colunas: {list(df_consolidado.columns)}")
    display(df_consolidado.head(10))

Total de registros: 5992
Período: 2025-07-07 até 2025-10-06
Colunas: ['date', 'open', 'high', 'low', 'close', 'volume', 'ticker']


,date,open,high,low,close,volume,ticker
0,2025-07-08 10:00:00,13.42,13.49,13.27,13.38,32803300.0,ABEV3
1,2025-07-09 10:00:00,13.45,13.45,13.23,13.31,31299800.0,ABEV3
2,2025-07-10 10:00:00,13.22,13.38,13.15,13.23,26865400.0,ABEV3
3,2025-07-11 10:00:00,13.19,13.38,13.08,13.33,40755100.0,ABEV3
4,2025-07-14 10:00:00,13.26,13.57,13.26,13.29,36420400.0,ABEV3
5,2025-07-15 10:00:00,13.37,13.52,13.23,13.27,27675300.0,ABEV3
6,2025-07-16 10:00:00,13.38,13.73,13.31,13.68,48903600.0,ABEV3
7,2025-07-17 10:00:00,13.67,13.69,13.44,13.46,19213600.0,ABEV3
8,2025-07-18 10:00:00,13.33,13.49,13.29,13.32,23902900.0,ABEV3
9,2025-07-21 10:00:00,13.35,13.45,13.26,13.42,21355000.0,ABEV3


## Exportar Resultados

In [4]:
# Salvar o DataFrame consolidado
if 'df_consolidado' in locals() and not df_consolidado.empty:
    export_dir = Path('../data/processed')
    export_dir.mkdir(parents=True, exist_ok=True)
    
    # Salvar como Parquet
    path_consolidado = export_dir / 'df_consolidado.parquet'
    df_consolidado.to_parquet(path_consolidado, index=False)
    
    # exportar como CSV
    csv_path = export_dir / 'df_consolidado.csv'
    df_consolidado.to_csv(csv_path, index=False)
    